In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection  import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test) 

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB

param_grid={'alpha':[1.0,5,10],'force_alpha':[True,False],'fit_prior':[True,False],'binarize':[0.0,None]}
grid=GridSearchCV(BernoulliNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [1.0, 5, 10], 'binarize': [0.0, None],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'alpha': 1.0, 'binarize': 0.0, 'fit_prior': False, 'force_alpha': True}: 0.7798880842659645


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[60 19]
 [ 8 33]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.88      0.76      0.82        79
           1       0.63      0.80      0.71        41

    accuracy                           0.78       120
   macro avg       0.76      0.78      0.76       120
weighted avg       0.80      0.78      0.78       120



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.8510342698363693

In [18]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026231,0.007915,0.015007,0.003963,1.0,0.0,True,True,"{'alpha': 1.0, 'binarize': 0.0, 'fit_prior': T...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,7
1,0.009909,0.004633,0.010819,0.003719,1.0,0.0,True,False,"{'alpha': 1.0, 'binarize': 0.0, 'fit_prior': T...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,7
2,0.007875,0.000817,0.013042,0.000657,1.0,0.0,False,True,"{'alpha': 1.0, 'binarize': 0.0, 'fit_prior': F...",0.754941,0.650298,0.702624,0.736088,0.789032,0.726597,0.047303,1
3,0.009086,0.003700,0.013841,0.002915,1.0,0.0,False,False,"{'alpha': 1.0, 'binarize': 0.0, 'fit_prior': F...",0.754941,0.650298,0.702624,0.736088,0.789032,0.726597,0.047303,1
4,0.010523,0.003144,0.011878,0.003410,1.0,None,True,True,"{'alpha': 1.0, 'binarize': None, 'fit_prior': ...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,13
5,0.008155,0.001984,0.008781,0.004635,1.0,None,True,False,"{'alpha': 1.0, 'binarize': None, 'fit_prior': ...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,13
6,0.009992,0.003259,0.013284,0.002752,1.0,None,False,True,"{'alpha': 1.0, 'binarize': None, 'fit_prior': ...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,13
7,0.007231,0.002978,0.011703,0.003514,1.0,None,False,False,"{'alpha': 1.0, 'binarize': None, 'fit_prior': ...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,13
8,0.007767,0.002405,0.012040,0.001109,5,0.0,True,True,"{'alpha': 5, 'binarize': 0.0, 'fit_prior': Tru...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,7
9,0.010989,0.004234,0.010923,0.002473,5,0.0,True,False,"{'alpha': 5, 'binarize': 0.0, 'fit_prior': Tru...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,7
